In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime
import xgboost as xgb
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor


import os


In [2]:
X = pd.read_csv("/kaggle/input/production-quality/data_X.csv")
submission = pd.read_csv("/kaggle/input/production-quality/sample_submission.csv")
Y = pd.read_csv("/kaggle/input/production-quality/data_Y.csv")

In [3]:
X["date_time"]=pd.to_datetime(X["date_time"])
X["date_hour"] = X["date_time"].apply(lambda x: x.strftime("%d-%m-%Y-%H"))

In [4]:
X

,date_time,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data,date_hour
0,2015-01-01 00:00:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22,01-01-2015-00
1,2015-01-01 00:01:00,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22,01-01-2015-00
2,2015-01-01 00:02:00,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22,01-01-2015-00
3,2015-01-01 00:03:00,213,211,211,344,351,346,477,473,481,355,349,355,242,241,242,167.34,9.22,01-01-2015-00
4,2015-01-01 00:04:00,213,211,211,343,350,346,478,473,482,358,349,355,243,241,242,163.04,9.22,01-01-2015-00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103836,2018-12-31 23:56:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,157.20,8.44,31-12-2018-23
2103837,2018-12-31 23:57:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,160.40,8.44,31-12-2018-23
2103838,2018-12-31 23:58:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,160.14,8.44,31-12-2018-23
2103839,2018-12-31 23:59:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,162.96,8.44,31-12-2018-23


In [5]:
# df = pd.DataFrame()
# for date_hour in list(X.date_hour.unique()):
#     temp = X.loc[X.date_hour==date_hour]
#     ah = list(temp["AH_data"].unique())[0]
#     temp = temp[temp.columns[1:]]
#     temp = pd.DataFrame(temp.set_index(["date_hour","AH_data"]).stack()).reset_index()[["level_2",0]].T.drop(["level_2"])
#     temp["date_hour"]= date_hour
#     temp["AH_data"]=ah
#     df = pd.concat([df,temp])

In [6]:
df = pd.read_csv("/kaggle/input/training-frame/training_frame.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,952,953,954,955,956,957,958,959,date_hour,AH_data
0,212.0,210.0,211.0,347.0,353.0,347.0,474.0,473.0,481.0,346.0,...,490.0,331.0,356.0,346.0,244.0,240.0,238.0,164.30,01-01-2015-00,9.22
1,210.0,213.0,212.0,299.0,300.0,346.0,523.0,473.0,490.0,330.0,...,490.0,366.0,351.0,342.0,239.0,237.0,238.0,163.35,01-01-2015-01,7.82
2,213.0,193.0,212.0,285.0,262.0,363.0,527.0,481.0,490.0,366.0,...,483.0,347.0,344.0,346.0,235.0,235.0,238.0,165.59,01-01-2015-02,6.03
3,248.0,207.0,212.0,303.0,271.0,331.0,486.0,488.0,482.0,347.0,...,481.0,333.0,361.0,354.0,239.0,241.0,238.0,164.89,01-01-2015-03,8.36
4,232.0,229.0,239.0,355.0,152.0,349.0,433.0,467.0,481.0,333.0,...,486.0,340.0,342.0,353.0,238.0,246.0,240.0,164.68,01-01-2015-04,7.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35060,264.0,260.0,257.0,377.0,363.0,378.0,452.0,449.0,447.0,334.0,...,484.0,328.0,321.0,323.0,277.0,280.0,278.0,186.07,31-12-2018-20,6.38
35061,257.0,274.0,255.0,373.0,368.0,365.0,482.0,449.0,484.0,328.0,...,496.0,325.0,327.0,326.0,277.0,277.0,280.0,164.91,31-12-2018-21,8.37
35062,322.0,286.0,256.0,352.0,343.0,349.0,481.0,449.0,496.0,325.0,...,491.0,325.0,332.0,328.0,277.0,276.0,280.0,158.21,31-12-2018-22,6.13
35063,271.0,252.0,262.0,357.0,359.0,360.0,481.0,449.0,491.0,325.0,...,491.0,325.0,328.0,328.0,277.0,276.0,280.0,162.96,31-12-2018-23,8.44


df is our transformed dataset. We can extract our training input and our submission input using date_hour.

In [7]:
Y["date_shifted"] = pd.to_datetime(Y["date_time"]) - datetime.timedelta(hours=1)
Y["date_shifted"] = pd.to_datetime(Y["date_shifted"])
Y["date_shifted"] = Y["date_shifted"].apply(lambda x: x.strftime("%d-%m-%Y-%H"))

In [8]:
training = pd.merge(df,Y[["date_shifted","quality"]],left_on="date_hour",right_on="date_shifted",how="inner")

In [9]:
submission["date_hour"] = pd.to_datetime(submission["date_time"]).apply(lambda x: x.strftime("%d-%m-%Y-%H"))
validation = pd.merge(df,submission[["date_hour","quality"]],left_on="date_hour",right_on="date_hour",how="inner")

In [10]:
training

,0,1,2,3,4,5,6,7,8,9,...,954,955,956,957,958,959,date_hour,AH_data,date_shifted,quality
0,265.0,337.0,263.0,335.0,326.0,340.0,493.0,497.0,527.0,316.0,...,376.0,337.0,231.0,236.0,243.0,155.41,03-01-2015-23,4.73,03-01-2015-23,392
1,276.0,330.0,272.0,323.0,335.0,351.0,505.0,501.0,655.0,325.0,...,395.0,336.0,233.0,241.0,230.0,159.57,04-01-2015-00,7.90,04-01-2015-00,384
2,277.0,260.0,272.0,320.0,333.0,356.0,500.0,502.0,700.0,336.0,...,399.0,333.0,242.0,250.0,238.0,158.89,04-01-2015-01,6.96,04-01-2015-01,393
3,265.0,218.0,261.0,325.0,336.0,333.0,504.0,499.0,443.0,347.0,...,388.0,333.0,255.0,258.0,247.0,155.87,04-01-2015-02,7.29,04-01-2015-02,399
4,243.0,237.0,252.0,327.0,330.0,308.0,519.0,498.0,537.0,342.0,...,372.0,343.0,263.0,263.0,260.0,195.59,04-01-2015-03,7.11,04-01-2015-03,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29179,256.0,249.0,260.0,381.0,389.0,384.0,459.0,326.0,465.0,358.0,...,351.0,349.0,235.0,254.0,253.0,157.34,03-05-2018-18,7.79,03-05-2018-18,438
29180,257.0,259.0,267.0,379.0,382.0,355.0,443.0,378.0,461.0,354.0,...,343.0,348.0,234.0,253.0,247.0,153.69,03-05-2018-19,7.33,03-05-2018-19,452
29181,259.0,255.0,259.0,373.0,369.0,363.0,433.0,466.0,453.0,361.0,...,359.0,345.0,255.0,256.0,246.0,156.31,03-05-2018-20,7.87,03-05-2018-20,455
29182,260.0,254.0,262.0,364.0,350.0,352.0,429.0,425.0,446.0,347.0,...,356.0,343.0,253.0,264.0,249.0,154.40,03-05-2018-21,7.65,03-05-2018-21,455


In [11]:
xgb_model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
dt_model = DecisionTreeRegressor(random_state=0)
X_val = training.drop(["date_hour","date_shifted","quality"],axis=1)
y_val = training["quality"]

To check how well model performs overall I will use cross-validation. 

In [12]:
xgb_cv =cross_val_score(xgb_model,X_val,y_val,cv=10,scoring=('neg_mean_absolute_error'))

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will gen

In [13]:
print("Average XGB Cross Validation MAE: {0}".format(np.abs(xgb_cv.mean())))
print("Best XGB Cross Validation MAE: {0}".format(np.abs(xgb_cv.max())))

Average XGB Cross Validation MAE: 7.279337593640449
Best XGB Cross Validation MAE: 7.012255248814341


In [14]:
cv_dt = cross_val_score(dt_model,X_val,y_val,cv=5,scoring=('neg_mean_absolute_error'))

In [15]:
print("Average Decision Tree Cross Validation MAE: {0}".format(np.abs(cv_dt.mean())))
print("Best Decision Tree Cross Validation MAE: {0}".format(np.abs(cv_dt.max())))

Average Decision Tree Cross Validation MAE: 13.35650716406634
Best Decision Tree Cross Validation MAE: 12.81891382559534


So after cross validation I can see that XGB performs better than DecisionTrees. I wouldn't assume that MAE = 7.012 is our final result as we used 90% of training data to train models, so we still can have a little bit more than that.

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X_val, y_val, test_size=0.15, random_state=0)

In [17]:
dt_model.fit(X_train,y_train)

DecisionTreeRegressor(random_state=0)

In [18]:
xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [19]:
predictions_xgb = xgb_model.predict(X_test)
predictions_dt = dt_model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [20]:
print("XGBoost Regressor MAE: {0}".format(mean_absolute_error(y_test,predictions_xgb)))
print("DecisionTree Regressor MAE: {0}".format(mean_absolute_error(y_test,predictions_dt)))

XGBoost Regressor MAE: 6.954141507904624
DecisionTree Regressor MAE: 12.894700776610325


Here we can see that MAE is sitting at 6.954.

In [21]:
results = pd.DataFrame()
results["True"] = y_test
results["Predicted XGB"]=predictions_xgb
results["Predicted DecisionTree"]=predictions_dt

In [22]:
results

,True,Predicted XGB,Predicted DecisionTree
20158,379,372.363007,369.0
23734,382,381.123718,382.0
12911,323,312.392517,309.0
7801,309,321.548492,296.0
21524,413,418.773315,409.0
...,...,...,...
5116,369,365.769714,367.0
2069,320,338.125671,344.0
27814,373,371.064575,382.0
18761,404,402.986237,414.0


## Submission file

In [23]:
model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model.fit(X_val,y_val)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
validation = validation.drop(["date_hour","quality"],axis=1)

In [25]:
quality = model.predict(validation)
submission["quality"]=quality

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [26]:
submission = submission[["date_time","quality"]]

In [27]:
submission

,date_time,quality
0,2018-05-04 00:05:00,442.543762
1,2018-05-04 01:05:00,431.179291
2,2018-05-04 02:05:00,418.005981
3,2018-05-04 03:05:00,409.189789
4,2018-05-04 04:05:00,410.348419
...,...,...
5803,2018-12-31 19:05:00,468.196594
5804,2018-12-31 20:05:00,460.236115
5805,2018-12-31 21:05:00,447.865356
5806,2018-12-31 22:05:00,442.773468
